In [3]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from filelock import FileLock
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as trans
import ray
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from ray.tune.schedulers import ASHAScheduler

In [4]:
def load_data(data_dir="./data"):
    transforms = trans.Compose[
        trans.toTensor(),
        trans.Normalize((0.5, 0.5, 0.5,), (0.5, 0.5, 0.5))
    ]

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transforms
        )
        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transforms
        )

    return trainset, testset

class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
def train_cifar(config):
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(os.path.join(loaded_checkpoint_dir, "checkpoint.pt"))

        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset)*0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.Dataloader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000==1999:
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1, running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        os.makedirs("my_model", exist_ok=True)
        torch.save(
            (net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report({"loss": (val_loss / val_steps), "accuracy": correct/total}, checkpoint=checkpoint)
        print("Finished training")

In [8]:
def test_best_model(best_result):
    best_trained_model = Net(best_result.config["l1"], best_result.config["l2"])
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(best_result.checkpoint.to_directory(), "checkpoint.pt")

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size()
            correct += (predicted == labels).sum().item()

    print("Best trial test set accuracy: {}".format(correct/total))

In [12]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    config = {
        "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16])
    }
    scheduler = ASHAScheduler(
        max_t = max_num_epochs,
        grace_period=1,
        reduction_factor=2
    )
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"cpu":2, "gpu":gpus_per_trial}
        ),
        tune_config=tune.TuneConfig(
            metric="loss",
            mode="min",
            scheduler=scheduler,
            num_samples=num_samples,
        ),
        param_space=config,
    )
    results = tuner.fit()

    best_result = results.get_best_result("loss", "min")

    print("Best trial config: {}".format(best_result.config))
    print("Best trial final validation loss: {}".format(
        best_result.metrics["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_result.metrics["accuracy"]))

    test_best_model(best_result)

In [13]:
main(num_samples=2, max_num_epochs=2, gpus_per_trial=0)

2023-03-06 16:44:01,971	INFO worker.py:1553 -- Started a local Ray instance.


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\juliu\miniconda3\envs\robustness_env\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\juliu\AppData\Local\Temp\ipykernel_19376\2662514348.py", line 1, in <module>
    main(num_samples=2, max_num_epochs=2, gpus_per_trial=0)
  File "C:\Users\juliu\AppData\Local\Temp\ipykernel_19376\3695509219.py", line 27, in main
    results = tuner.fit()
  File "C:\Users\juliu\miniconda3\envs\robustness_env\lib\site-packages\ray\tune\tuner.py", line 292, in fit
    return self._local_tuner.fit()
  File "C:\Users\juliu\miniconda3\envs\robustness_env\lib\site-packages\ray\tune\impl\tuner_internal.py", line 455, in fit
    analysis = self._fit_internal(trainable, param_space)
  File "C:\Users\juliu\miniconda3\envs\robustness_env\lib\site-packages\ray\tune\impl\tuner_internal.py", line 572, in _fit_internal
    analysis = run(
  File "C:\Users\juliu\miniconda

In [29]:
from __future__ import print_function

import argparse
import os
import torch
import torch.optim as optim
from torchvision import transforms, datasets

import ray
from ray import air, tune
from ray.tune.schedulers import ASHAScheduler, AsyncHyperBandScheduler

In [30]:
EPOCH_SIZE = 256
TEST_SIZE = 128

In [31]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

In [32]:
def train(model, optimizer, train_loader, device=None):
    device = device or torch.device("cpu")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if batch_idx * len(data) > EPOCH_SIZE:
            return
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

def test(model, data_loader, device=None):
    device = device or torch.device("cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            if batch_idx * len(data) > TEST_SIZE:
                break
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return correct / total

def get_data_loaders(batch_size=64):
    mnist_transforms = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )

    # We add FileLock here because multiple workers will want to
    # download data, and this may cause overwrites since
    # DataLoader is not threadsafe.
    with FileLock(os.path.expanduser("~/data.lock")):
        train_loader = torch.utils.data.DataLoader(
            datasets.MNIST(
                "~/data", train=True, download=True, transform=mnist_transforms
            ),
            batch_size=batch_size,
            shuffle=True,
        )
        test_loader = torch.utils.data.DataLoader(
            datasets.MNIST(
                "~/data", train=False, download=True, transform=mnist_transforms
            ),
            batch_size=batch_size,
            shuffle=True,
        )
    return train_loader, test_loader

In [33]:
class TrainMNIST(tune.Trainable):
    def setup(self, config):
        use_cuda = True if torch.cuda.is_available() else False
        self.device = torch.device("cuda" if use_cuda else "cpu")
        self.train_loader, self.test_loader = get_data_loaders()
        self.model = ConvNet().to(self.device)
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.01, momentum=0.9)

    def step(self):
        train(self.model, self.optimizer, self.train_loader, device=self.device)
        acc = test(self.model, self.test_loader, self.device)
        return{"mean_accuracy": acc}

    def save_checkpoint(self, checkpoint_dir):
        checkpoint_path = os.path.join(checkpoint_dir, "model.pth")
        torch.save(self.model.state_dict(), checkpoint_path)
        return checkpoint_path

    def load_checkpoint(self, checkpoint_path):
        self.model.load_state_dict(torch.load(checkpoint_path))

In [38]:
ray.shutdown()

In [39]:
ray.init(num_cpus=0)
sched = ASHAScheduler()

tuner = tune.Tuner(
    tune.with_resources(TrainMNIST, resources={"cpu": 3, "gpu": 0}),
    run_config=air.RunConfig(
        stop={
            "mean_accuracy": 0.95,
            "training_iteration": 20
        },
        # checkpoint_config=air.CheckpointConfig(
        #     checkpoint_at_end=True,
        #     checkpoint_frequency=3
        # ),
    ),
    tune_config=tune.TuneConfig(
        metric="mean_accuracy",
        mode="max",
        scheduler=sched,
        num_samples=20
    ),
    param_space={
        "lr": tune.uniform(0.001, 0.1),
        "momentum": tune.uniform(0.1, 0.9),
    },
)

2023-03-07 15:14:03,694	INFO worker.py:1553 -- Started a local Ray instance.


In [40]:
results = tuner.fit()
print("Best config is:", results.get_best_result().config)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\juliu\miniconda3\envs\robustness_env\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\juliu\AppData\Local\Temp\ipykernel_19376\2372888358.py", line 1, in <module>
    results = tuner.fit()
  File "C:\Users\juliu\miniconda3\envs\robustness_env\lib\site-packages\ray\tune\tuner.py", line 292, in fit
    return self._local_tuner.fit()
  File "C:\Users\juliu\miniconda3\envs\robustness_env\lib\site-packages\ray\tune\impl\tuner_internal.py", line 455, in fit
    analysis = self._fit_internal(trainable, param_space)
  File "C:\Users\juliu\miniconda3\envs\robustness_env\lib\site-packages\ray\tune\impl\tuner_internal.py", line 572, in _fit_internal
    analysis = run(
  File "C:\Users\juliu\miniconda3\envs\robustness_env\lib\site-packages\ray\tune\tune.py", line 678, in run
    callbacks = _create_default_callbacks(
  File "C:\Users\juliu\miniconda